In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics


import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv('diabetes2.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
# df = df.dropna()
# df.info()

df['Glucose'].fillna(df['Glucose'].mean(), inplace = True)
df['BloodPressure'].fillna(df['BloodPressure'].mean(), inplace = True)
df['SkinThickness'].fillna(df['SkinThickness'].mean(), inplace = True)
df['Insulin'].fillna(df['Insulin'].mean(), inplace = True)
df['BMI'].fillna(df['BMI'].mean(), inplace = True)

In [ ]:
price_correlation = (df.corr()['price']).sort_values(key=abs)
print(price_correlation)
df.drop(['BloodPressure','DiabetesPedigreeFunction'], inplace = True, axis = 1)

In [ ]:
df['cut'].unique()
df = pd.get_dummies(df, columns=['cut', 'color', 'clarity'])
df.replace({False: 0, True: 1}, inplace=True)
df.info()

In [ ]:
def remove_outliers(df,column):
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR = Q3 - Q1
  lower = Q1 - 1.5*IQR
  upper = Q3 + 1.5*IQR
  return df[(df[column] >= lower) & (df[column] <= upper)]
#outliers have been removed
outlier_columns = ['max heart rate','oldpeak']
for column in outlier_columns:
  df2 = remove_outliers(df1,column)
df2.reset_index(drop=True, inplace=True)

In [ ]:
columns_to_scale = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
numeric_data = df[columns_to_scale]
scaler = StandardScaler()
scaler.fit(numeric_data)
scaled_data = scaler.transform(numeric_data)
df[columns_to_scale] = scaled_data

In [ ]:
df = df.sample(frac=1, random_state=50)

In [ ]:
y = df['Outcome']
X = df.drop('Outcome', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 50)

print(X_train.shape)
print(y_train.shape)

In [ ]:
classifiers = {
    # 'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=50),
    'Logistic Regression': LogisticRegression()
}

# Define hyperparameters grid for each classifier
# param_grids = {
#     'SVM': {
#         'C': [0.1, 1, 10],
#         'gamma': ['scale', 'auto'],
#         'kernel': ['rbf', 'linear']
#     },
#     'KNN': {
#         'n_neighbors': [3, 5, 7],
#         'weights': ['uniform', 'distance']
#     },
#     'Random Forest': {
#         'n_estimators': [50, 100],
#         'max_depth': [None, 10],
#         'min_samples_split': [2, 5],
#         'max_features': ['sqrt', 'log2']
#     },
#     'Logistic Regression': {
#         'solver': ['saga', 'liblinear'],
#         'penalty': ['l1', 'l2'],
#         'C': [0.1, 1, 10],
#         'max_iter': [100, 500]
#     }
# }

param_grids = {
    # 'SVM': {
    #     'C': [0.1, 1, 10, 100, 1000],
    #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'],
    #     'kernel': ['rbf','linear']
    # },
    'KNN': {
        'n_neighbors': [3,4,5,6,7,8,9,10],
        'weights': ['uniform', 'distance']
    },
    'Random Forest': {
      'n_estimators': [50, 100, 150],
      'max_depth': [None, 10, 20],
      'min_samples_split': [2, 5, 10],
      'max_features': ['sqrt', 'log2']
    },
    # 'HistGradientBoosting': {
    #     'learning_rate': [0.1, 0.01, 0.001],
    #     'max_iter': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    # },
    'Logistic Regression':{'solver': ['saga', 'liblinear'],
                  'penalty': ['l1','l2'],
                 'C': [0.001,0.01,0.1,1,10,100,1000],
                  'max_iter' : [10,100,500]}
}


# param_grids = {
#     'SVM': {
#         'C': [0.1],
#         'gamma': ['scale', 'auto'],
#         'kernel': ['rbf', 'linear']
#     },
#     'KNN': {
#         'n_neighbors': [3, 5, 7],
#         'weights': ['uniform', 'distance']
#     },
#     'Random Forest': {
#         'n_estimators': [50, 100],
#         'max_depth': [None, 10],
#         'min_samples_split': [2, 5],
#         'max_features': ['sqrt', 'log2']
#     },
#     'Logistic Regression': {
#         'solver': ['saga', 'liblinear'],
#         'penalty': ['l1', 'l2'],
#         'C': [0.1, 1, 10],
#         'max_iter': [100, 500]
#     }
# }

In [ ]:
results = {}
for name, classifier in classifiers.items():
    print("Training: ", name)
    gridSearch = GridSearchCV(estimator=classifier, param_grid=param_grids[name], cv=5, n_jobs=-1)
    gridSearch.fit(X_train, y_train)
    results[name] = gridSearch.best_estimator_

print(results)

In [ ]:
for name, clf in results.items():
    print("\nResults for", name)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

In [ ]:
parameters = {
    'fit_intercept': [True, False],
    'positive': [True, False]
}
gridSearch = GridSearchCV(estimator=LinearRegression(), param_grid=parameters, cv=5, n_jobs=-1)
gridSearch.fit(X_train, y_train)
bestModel = gridSearch.best_estimator_
y_pred = bestModel.predict(X_test)
bestModel_score = bestModel.score(X_test, y_test)
print(bestModel_score)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
model = Sequential()

# add input layer size to the first hidden layer
model.add(Dense(256, input_shape=(784,), activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation="relu"))
# output layer
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
learning_rate = 0.01

# stochastic gradient descent optimizer
sgd_optimizer = keras.optimizers.Adam()
# categoricall crossentropy loss function
cc_loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=sgd_optimizer, loss=cc_loss, metrics=["accuracy"])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)


y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
encoder = OneHotEncoder(sparse_output=False).fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=64)
loss, accuracy = model.evaluate(X_test, y_test, batch_size=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = np.argmax(y_pred, axis = 1)
correct = np.argmax(y_test, axis = 1)
# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(correct, predictions))

# Compute additional metrics
print("\nClassification Report:")
print(classification_report(correct, predictions))